## Main script to perform heart rate estimation of wearable PPG

This script uses both PPG and accelerometer and performs the following steps:
1. Loading all metadata of PPG and IMU
2. Query on data availability + synchronization
3. Loading relevant segment sensor data using tsdf wrapper (start for loop over synchronized segment indices)
4. Synchronize the data (correct indices etc)
5. Data preprocessing
6. Feature extraction
7. Classification


## Architecture overview
The script implements the following steps:
 - Step 1: IMU and PPG preprocessing
 - Step 2: IMU and PPG feature extraction
 - Step 3: Signal quality assessment

In [93]:
# Automatically reload modules
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
from datetime import datetime
from pathlib import Path
import json
import os
from typing import List

import tsdf
import dbpd
from dbpd.constants import DataColumns
from dbpd.ppg_preprocessing import tsdf_scan_meta, synchronization, extract_overlapping_segments


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [94]:
# Cell has the tag 'parameters'
path_to_data =  '../../../tests/data'

input_path_imu = os.path.join(path_to_data, '1.sensor_data', 'imu')
input_path_ppg = os.path.join(path_to_data, '1.sensor_data', 'ppg')
output_path = os.path.join(path_to_data, '2.preprocessed_data', 'ppg')

In [95]:
# Constants
sensor_imu = 'IMU'
sensor_ppg = 'PPG'

imu_meta_filename = f'{sensor_imu}_meta.json'
imu_values_filename = f'{sensor_imu}_samples.bin'
imu_time_filename = f'{sensor_imu}_time.bin'

ppg_meta_filename = f'{sensor_ppg}_meta.json'
ppg_values_filename = f'{sensor_ppg}_samples.bin'
ppg_time_filename = f'{sensor_ppg}_time.bin'

rotation_units = 'deg/s'
acceleration_units = 'm/s^2'

d_channels_units = {
    DataColumns.ACCELEROMETER_X: acceleration_units,
    DataColumns.ACCELEROMETER_Y: acceleration_units,
    DataColumns.ACCELEROMETER_Z: acceleration_units,
    DataColumns.GYROSCOPE_X: rotation_units,
    DataColumns.GYROSCOPE_Y: rotation_units,
    DataColumns.GYROSCOPE_Z: rotation_units,

}

# filtering
sampling_frequency = 100
lower_cutoff_frequency = 0.3
filter_order = 4

# Constants
UNIX_TICKS_MS = 1000.0
FS_PPG = 30  # Sampling rate for PPG
FS_IMU = 100  # Sampling rate for IMU

## 1. Loading all metadata of PPG and IMU

In [96]:
meta_ppg = tsdf_scan_meta(input_path_ppg)
meta_imu = tsdf_scan_meta(input_path_imu)

## 2. Query on data availability + synchronization

Calculate PPG and IMU segments that describe the same data

In [97]:
segment_ppg, segment_imu = synchronization(meta_ppg, meta_imu)  # Define `synchronization`

## 3. Loading relevant segment sensor data

In [98]:
n = 0  # Assuming we're only looking at the first synchronized segment pair - the loop here should be over all segment pairs
meta_path_ppg = meta_ppg[segment_ppg[n]]['tsdf_meta_fullpath']
meta_path_imu = meta_imu[segment_imu[n]]['tsdf_meta_fullpath']

metadata_list_ppg = tsdf.load_metadata_from_path(meta_path_ppg)
metadata_list_imu = tsdf.load_metadata_from_path(meta_path_imu)

In [99]:
# Load PPG data
ppg_metadata_time = metadata_list_ppg[ppg_time_filename]
ppg_metadata_samples = metadata_list_ppg[ppg_values_filename]
df_ppg = tsdf.load_dataframe_from_binaries([ppg_metadata_time, ppg_metadata_samples], tsdf.constants.ConcatenationType.columns)

# Load IMU data
imu_metadata_time = metadata_list_imu[imu_time_filename]
imu_metadata_samples = metadata_list_imu[imu_values_filename]
df_imu = tsdf.load_dataframe_from_binaries([imu_metadata_time, imu_metadata_samples], tsdf.constants.ConcatenationType.columns)

In [100]:
df_ppg.head(2)

,time,green
0,0.000000,649511
1,9.959961,648214


In [101]:
# Drop the gyroscope columns from the IMU data
cols_to_drop = df_imu.filter(regex='^rotation_').columns
df_imu.drop(cols_to_drop, axis=1, inplace=True)

df_imu = df_imu.rename(columns={f'acceleration_{a}': f'accelerometer_{a}' for a in ['x', 'y', 'z']})

df_imu.head(2)

,time,accelerometer_x,accelerometer_y,accelerometer_z
0,0.000000,-1151,1200,-572
1,10.040039,-1120,1303,-532


In [106]:
from dbpd.constants import DataColumns, TimeUnit

# Extract indices for time and samples (TSDF should support extracting data based on the channel names)
time_idx_ppg = metadata_list_ppg["PPG_time.bin"]
time_idx_imu = metadata_list_imu["IMU_time.bin"]
values_idx_ppg = metadata_list_ppg["PPG_samples.bin"]
values_idx_imu = metadata_list_imu["IMU_samples.bin"]

start_time_ppg = datetime.strptime(time_idx_ppg.start_iso8601, '%d-%b-%Y %H:%M:%S %Z').timestamp()
df_imu['time'] = dbpd.imu_preprocessing.transform_time_array(
    time_array=df_imu['time'],
    scale_factor=1000, 
    input_unit_type = TimeUnit.difference_ms,
    output_unit_type = TimeUnit.absolute_ms,
    start_time = start_time_ppg)

start_time_imu = datetime.strptime(time_idx_imu.start_iso8601, '%d-%b-%Y %H:%M:%S %Z').timestamp()
df_ppg['time'] = dbpd.imu_preprocessing.transform_time_array(
    time_array=df_ppg['time'],
    scale_factor=1000, 
    input_unit_type = TimeUnit.difference_ms,
    output_unit_type = TimeUnit.absolute_ms,
    start_time = start_time_imu)


df_imu.tail(2)
# ts_ppg = int(datetime_ppg.timestamp() * UNIX_TICKS_MS)
# ts_imu = int(datetime_imu.timestamp() * UNIX_TICKS_MS)

# # Calculating continuous time vectors
# t_ppg = np.cumsum(data_list_ppg[time_idx_ppg]) + ts_ppg
# t_imu = np.cumsum(data_list_imu[time_idx_imu]) + ts_imu

,time,accelerometer_x,accelerometer_y,accelerometer_z
72945,4.443100e+12,-502,-1075,-1689
72946,4.443220e+12,-509,-1068,-1689


In [108]:
# 4. Data synchronization on right indices
print("Shape of the original data:")
print(df_ppg.shape, df_imu.shape)
df_ppg_overlapping, df_imu_overlapping = extract_overlapping_segments(df_ppg, df_imu)
print("Shape of the overlapping segments:")
print(df_ppg_overlapping.shape, df_imu_overlapping.shape)

df_imu_overlapping.tail(5)


Shape of the original data:
(64775, 2) (72947, 4)
Shape of the overlapping segments:
(64775, 2) (64774, 4)


,time,accelerometer_x,accelerometer_y,accelerometer_z
64769,3.515070e+12,-1485,31,-1026
64770,3.515177e+12,-1470,-32,-1016
64771,3.515284e+12,-1479,-64,-1050
64772,3.515391e+12,-1484,-75,-1095
64773,3.515498e+12,-1498,-53,-1131


In [ ]:

# 5. Data preprocessing
# Implement `preprocessing_ppg` and `preprocessing_imu` to suit your data format
v_ppg_pre, tr_ppg_pre = preprocessing_ppg(v_ppg, FS_PPG)
v_imu_pre, tr_imu_pre = preprocessing_imu(v_imu, FS_IMU)

# Save preprocessed data
location = "../../tests/data/2.preprocessed_data/ppg"
save_preprocessed_data(v_ppg_pre, tr_ppg_pre, v_imu_pre, tr_imu_pre, location)

# Feature extraction and Classification
# Assume feature extraction and classification functions are implemented
features_ppg, features_imu = extract_features(v_ppg_pre, v_imu_pre)
classification_results = classify_signals(features_ppg, features_imu)

# Save the classification results
save_classification_data(classification_results, location)

# We need to implement:
# - synchronization: to find overlapping segments between PPG and IMU data based on metadata
# - extract_overlapping_segments: to calculate the correct indices for synchronized data segments
# - preprocessing_ppg, preprocessing_imu: functions to preprocess the raw PPG and IMU data
# - extract_features: to extract relevant features from the preprocessed data
# - classify_signals: to perform the classification on the extracted features
# - save_preprocessed_data, save_classification_data: functions to save data to files in a suitable format


NameError: name 'synchronization' is not defined